## Rubric
 
| Criteria                  | Ratings                                                                                                                                                                                             | Pts    |
| ------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------ |
| Doubly Linkedlist reverse | - **20 pts** Full Marks  <br> - **15 pts** tail not set <br> - **0 pts** No Marks                                                                                                                   | 20 pts |
| Linkedlist rotation       | - **20 pts** Full Marks  <br> - **15 pts** not efficient (works but not the most efficient) <br> - **10 pts** does not work for some cases                                                          | 20 pts |
| Linkedlist union          | - **20 pts** Full Marks <br> - **15 pts** does not work for some cases or is inefficient (must be efficient in time and space) <br> - **0 pts** No Marks                                            | 20 pts |
| Tree to linkedlist        | - **20 pts** Full Marks <br> - **15 pts** does not work for some cases or is inefficient (must be efficient in time and space) <br> - **10 pts** Must write two functions <br> - **0 pts** No Marks | 20 pts |
| Largest tree width        | - **20 pts** Full Marks <br> - **15 pts** does not work for some cases or is inefficient (must be efficient in time and space) <br> - **0 pts** No Marks                                            | 20 pts |

**Total Points: 100**

---

## 5. Largest Width of a Binary Tree

Write a function that takes a valid binary tree and returns its largest width.

- The largest width a binary tree is the highest number of nodes present at any single level
- It is not based on the number of potential node positions, only the actual nodes present at that level

Example: Using the same binary tree in #4, this function should return 3 (the last level has 3 nodes)

## Custom Libraries

Import custom libraries for data structures used in this assignment.

In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Protocol, Any, Generic, Callable, Optional, TypeVar, Iterator, List

T = TypeVar("T")


# A protocol expressing that a type supports equality comparisons
class Comparable(Protocol):
    def __lt__(self, other: Any) -> bool: ...
    def __gt__(self, other: Any) -> bool: ...
    def __ge__(self, other: Any) -> bool: ...
    def __le__(self, other: Any) -> bool: ...
    def __eq__(self, other: Any) -> bool: ...


C = TypeVar("C", bound=Comparable)


# -----------------------------
# Node Base Classes
# -----------------------------
class NodeBase(Generic[T]):
    """Base class for nodes in linked lists."""

    def __init__(self, value: T):
        self.value: T = value


# -----------------------------
# Singly Linked List Node
# -----------------------------
class SLLNode(NodeBase[T]):
    """Singly-linked list node."""

    def __init__(self, value: T, next: Optional[SLLNode[T]] = None):
        super().__init__(value)
        self.next: Optional[SLLNode[T]] = next

    def __repr__(self) -> str:
        return f"SLLNode(value={self.value}, next={getattr(self.next, 'value', None)})"


# -----------------------------
# Doubly Linked List Node
# -----------------------------
class DLLNode(NodeBase[T]):
    """Doubly-linked list node."""

    def __init__(
        self,
        value: T,
        prev: Optional[DLLNode[T]] = None,
        next: Optional[DLLNode[T]] = None,
    ):
        super().__init__(value)
        self.prev: Optional[DLLNode[T]] = prev
        self.next: Optional[DLLNode[T]] = next

    def __repr__(self) -> str:
        return (
            f"DLLNode(value={self.value}, "
            f"prev={getattr(self.prev, 'value', None)}, "
            f"next={getattr(self.next, 'value', None)})"
        )


# -----------------------------
# Binary Tree Node
# -----------------------------
class BinaryTreeNode(NodeBase[C]):
    """A node in a binary tree, inheriting from NodeBase."""

    __slots__ = ("value", "left", "right")

    def __init__(
        self,
        value: C,
        left: Optional[BinaryTreeNode[C]] = None,
        right: Optional[BinaryTreeNode[C]] = None,
    ) -> None:
        super().__init__(value)
        self.left = left
        self.right = right

    def __repr__(self) -> str:
        return f"BinaryTreeNode({self.value!r})"


# -----------------------------
# Abstract Linked List Base
# -----------------------------
class LinkedListBase(ABC, Generic[T]):
    """Abstract base class for linked lists."""

    @property
    @abstractmethod
    def head(self) -> Optional[NodeBase[T]]:
        """Return the head node (first node)."""
        pass

    @abstractmethod
    def append(self, value: T) -> None:
        pass

    @abstractmethod
    def prepend(self, value: T) -> None:
        pass

    @abstractmethod
    def remove(self, value: T) -> bool:
        pass

    @abstractmethod
    def pop_left(self) -> T:
        pass

    @abstractmethod
    def clear(self) -> None:
        pass

    @abstractmethod
    def to_list(self) -> List[T]:
        pass

    @abstractmethod
    def __len__(self) -> int:
        pass

    @abstractmethod
    def __iter__(self) -> Iterator[T]:
        pass

    @abstractmethod
    def is_empty(self) -> bool:
        pass

    @abstractmethod
    def __repr__(self) -> str:
        pass


# -----------------------------
# Singly Linked List
# -----------------------------
class SinglyLinkedList(LinkedListBase[T]):
    """Singly-linked list implementation."""

    def __init__(self) -> None:
        self._head: Optional[SLLNode[T]] = None
        self._size: int = 0

    # Property to comply with LinkedListBase
    @property
    def head(self) -> Optional[SLLNode[T]]:
        return self._head

    @head.setter
    def head(self, node: Optional[SLLNode[T]]) -> None:
        self._head = node

    def __len__(self) -> int:
        return self._size

    def __iter__(self) -> Iterator[T]:
        current = self._head
        while current:
            yield current.value
            current = current.next

    def __repr__(self) -> str:
        values = " -> ".join(repr(v) for v in self)
        return f"SinglyLinkedList([{values}])"

    def is_empty(self) -> bool:
        return self._size == 0

    def append(self, value: T) -> None:
        new_node = SLLNode(value)
        if not self._head:
            self._head = new_node
        else:
            current = self._head
            while current.next:
                current = current.next
            current.next = new_node
        self._size += 1

    def prepend(self, value: T) -> None:
        self._head = SLLNode(value, next=self._head)
        self._size += 1

    def remove(self, value: T) -> bool:
        if not self._head:
            return False
        if self._head.value == value:
            self._head = self._head.next
            self._size -= 1
            return True
        prev = self._head
        curr = self._head.next
        while curr:
            if curr.value == value:
                prev.next = curr.next
                self._size -= 1
                return True
            prev, curr = curr, curr.next
        return False

    def pop_left(self) -> T:
        if not self._head:
            raise IndexError("pop from empty list")
        value = self._head.value
        self._head = self._head.next
        self._size -= 1
        return value

    def clear(self) -> None:
        self._head = None
        self._size = 0

    def to_list(self) -> List[T]:
        return list(iter(self))


# -----------------------------
# Doubly Linked List
# -----------------------------
class DoublyLinkedList(LinkedListBase[T]):
    """Doubly-linked list implementation."""

    def __init__(self) -> None:
        self._head: Optional[DLLNode[T]] = None
        self._tail: Optional[DLLNode[T]] = None
        self._size: int = 0

    # Property to comply with LinkedListBase
    @property
    def head(self) -> Optional[DLLNode[T]]:
        return self._head

    @head.setter
    def head(self, node: Optional[DLLNode[T]]) -> None:
        self._head = node

    @property
    def tail(self) -> Optional[DLLNode[T]]:
        return self._tail

    @tail.setter
    def tail(self, node: Optional[DLLNode[T]]) -> None:
        self._tail = node

    def __len__(self) -> int:
        return self._size

    def __iter__(self) -> Iterator[T]:
        current = self._head
        while current:
            yield current.value
            current = current.next

    def __repr__(self) -> str:
        if self.is_empty():
            return "DoublyLinkedList([])"
        values = " ⇄ ".join(repr(v) for v in self)
        return f"HEAD ⇄ {values} ⇄ TAIL"

    def is_empty(self) -> bool:
        return self._size == 0

    def append(self, value: T) -> None:
        new_node = DLLNode(value, prev=self._tail)
        if not self._head:
            self._head = new_node
        else:
            assert self._tail is not None
            self._tail.next = new_node
        self._tail = new_node
        self._size += 1

    def prepend(self, value: T) -> None:
        new_node = DLLNode(value, next=self._head)
        if self._head:
            self._head.prev = new_node
        else:
            self._tail = new_node
        self._head = new_node
        self._size += 1

    def remove(self, value: T) -> bool:
        current = self._head
        while current:
            if current.value == value:
                if current.prev:
                    current.prev.next = current.next
                else:
                    self._head = current.next
                if current.next:
                    current.next.prev = current.prev
                else:
                    self._tail = current.prev
                self._size -= 1
                return True
            current = current.next
        return False

    def pop_left(self) -> T:
        if not self._head:
            raise IndexError("pop from empty list")
        value = self._head.value
        self._head = self._head.next
        if self._head:
            self._head.prev = None
        else:
            self._tail = None
        self._size -= 1
        return value

    def pop(self) -> T:
        if not self._tail:
            raise IndexError("pop from empty list")
        value = self._tail.value
        self._tail = self._tail.prev
        if self._tail:
            self._tail.next = None
        else:
            self._head = None
        self._size -= 1
        return value

    def clear(self) -> None:
        self._head = None
        self._tail = None
        self._size = 0

    def to_list(self) -> List[T]:
        return list(iter(self))


class BinaryTree(Generic[C]):
    """Binary tree wrapper class."""

    __slots__ = ("_root", "_size")

    def __init__(self, root: Optional[BinaryTreeNode[C]] = None) -> None:
        self._root = root
        self._size = 0 if root is None else self._compute_size(root)

    @property
    def root(self) -> Optional[BinaryTreeNode[C]]:
        return self._root

    @property
    def size(self) -> int:
        return self._size

    def is_empty(self) -> bool:
        return self._root is None

    def _compute_size(self, node: Optional[BinaryTreeNode[C]]) -> int:
        if node is None:
            return 0
        return 1 + self._compute_size(node.left) + self._compute_size(node.right)

    # -------------------------------
    # Insert (simple binary search tree insert)
    # -------------------------------
    def insert(self, value: C) -> None:
        """Insert a value into the binary tree (BST style)."""

        def _insert(node: Optional[BinaryTreeNode[C]], value: C) -> BinaryTreeNode[C]:
            if node is None:
                return BinaryTreeNode(value)
            if value < node.value:
                node.left = _insert(node.left, value)
            else:
                node.right = _insert(node.right, value)
            return node

        self._root = _insert(self._root, value)
        self._size += 1

    # -------------------------------
    # Traversals
    # -------------------------------
    def inorder(self, visit: Callable[[C], None]) -> None:
        def _inorder(node: Optional[BinaryTreeNode[C]]):
            if node:
                _inorder(node.left)
                visit(node.value)
                _inorder(node.right)

        _inorder(self._root)

    def preorder(self, visit: Callable[[C], None]) -> None:
        def _preorder(node: Optional[BinaryTreeNode[C]]):
            if node:
                visit(node.value)
                _preorder(node.left)
                _preorder(node.right)

        _preorder(self._root)

    def postorder(self, visit: Callable[[C], None]) -> None:
        def _postorder(node: Optional[BinaryTreeNode[C]]):
            if node:
                _postorder(node.left)
                _postorder(node.right)
                visit(node.value)

        _postorder(self._root)

    # -------------------------------
    # Utility
    # -------------------------------
    def to_list(self) -> str:
        values: list[C] = []
        self.inorder(values.append)
        return values
        
    def __repr__(self) -> str:
        return f"BinaryTree({self.to_list()})"
        
    def pretty_print(self) -> str:
        if not self._root:
            return "<empty>"

        def _display(
            node: Optional["BinaryTreeNode[C]"],
        ) -> tuple[list[str], int, int, int]:
            """
            Returns:
                lines: list of strings for this subtree
                width: total width
                height: total height
                middle: horizontal position of root
            """
            if node is None:
                return [], 0, 0, 0

            line = f"┌{node.value}┐"
            width = len(line)

            if node.left is None and node.right is None:
                return [line], width, 1, width // 2

            # Recursively display left and right
            left_lines, left_width, left_height, left_middle = _display(node.left)
            right_lines, right_width, right_height, right_middle = _display(node.right)

            # Compute new width and height
            height = max(left_height, right_height) + 2
            middle = left_width + width // 2 + 1 if node.left else width // 2

            # Pad left/right lines
            left_lines += [" " * left_width] * (right_height - left_height)
            right_lines += [" " * right_width] * (left_height - right_height)

            # Combine lines
            lines = [" " * left_width + line + " " * right_width]  # root line
            for l, r in zip(left_lines, right_lines):
                lines.append(l + " " * width + r)
            return lines, left_width + width + right_width, height, middle

        lines, _, _, _ = _display(self._root)
        return "\n".join(lines)

        


## 1. Reverse a Doubly Linked List

Write a function to reverse the elements in a doubly linked list. Do not simply print it out, replace the values or use methods. It must have the references correctly set in reversed order.   
Also, write a print method – this should help with debugging.

---
### Answers

The below function `reverse_dll` reverses a doubly-linked list(`DoublyLinkedList`) in-place, with a time complexity of $O(n)$ and a space complexity of $O(1)$. It swaps the references of node object's next and previous values, while traversing the list. At the end, it simply swaps the head and tail references to maintain the correct structure. The test functions below validate some simple cases of this function.

Also provided is a `reverse_dll_half` function, which is done for educational purposes and should **not** be considered for this assignment. Instead of updating node next and previous references, it simply swaps the mirrored-values within the list. This is made possible by including both head and tail references within the list, and incrementing/decrementing those respectively. This is similar to an array-based reversal, but is much less efficient since the doubly-linked list is not indexed. Furthermore, it is **not stable** for use in applications which rely on pointer references to node objects (e.g., some sorting algorithms which assume that the node pointers are ordered), since this implementation only swaps out the value data within each object and can break ordering requirements.

In [2]:
def reverse_dll(dll: DoublyLinkedList[T]):
    '''
    Reverses a doubly-linked list in-place.
    Time Complexity: O(n) since we must traverse the doubly-linked list
    Space Complexity: O(1) since all operations are done in-place
    '''

    # This also handles empty or single-element list
    curr = dll.head
    while curr: # O(n)
        curr.next, curr.prev = curr.prev, curr.next # O(1)
        curr = curr.prev # move forward (old prev was swapped with next)

    # swap head and tail refs
    dll.head, dll.tail = dll.tail, dll.head # O(1)

def reverse_dll_half(dll: DoublyLinkedList[T]) -> DoublyLinkedList[T]:
    '''
    Reverses a doubly linked list by swapping values
    from head and tail in n//2 iterations.
    NOTE: this may not be stable if applications need pointer references
    to nodes, since the pointers will now point to new data in the same
    object. This is not a "true" reversal, but is done for educational 
    purposes.
    Time Complexity: O(n/2) = O(n)
    Space Complexity: O(1)
    '''
    left = dll.head
    right = dll.tail

    for _ in range(len(dll) // 2):
        left.value, right.value = right.value, left.value
        left = left.next
        right = right.prev

    return dll   

In [3]:
def build_list(values: List[int]) -> DoublyLinkedList[int]:
    dll = DoublyLinkedList[int]()
    for v in values:
        dll.append(v)
    return dll


def test_reverse_dll():
    test_cases: List[Tuple[List[int], List[int]]] = [
        ([], []),                          # empty list
        ([1], [1]),                        # single element
        ([1, 2], [2, 1]),                  # two elements
        ([1, 2, 3], [3, 2, 1]),            # odd length
        ([10, 20, 30, 40], [40, 30, 20, 10]),  # even length
    ]

    for i, (input_values, expected_values) in enumerate(test_cases, start=1):
        dll = build_list(input_values)
        reverse_dll(dll)

        result = list(dll)
        head_val = dll.head.value if dll.head else None
        tail_val = dll.tail.value if dll.tail else None

        print(f"\n[TEST {i}]")
        print(f"Input:          {input_values}")
        print(f"Expected:       {expected_values}")
        print(f"Got:            {result}")
        print(f"Head -> Tail:   {head_val} ... {tail_val}")
        print("PASS ✅" if result == expected_values else "FAIL ❌")

test_reverse_dll()


[TEST 1]
Input:          []
Expected:       []
Got:            []
Head -> Tail:   None ... None
PASS ✅

[TEST 2]
Input:          [1]
Expected:       [1]
Got:            [1]
Head -> Tail:   1 ... 1
PASS ✅

[TEST 3]
Input:          [1, 2]
Expected:       [2, 1]
Got:            [2, 1]
Head -> Tail:   2 ... 1
PASS ✅

[TEST 4]
Input:          [1, 2, 3]
Expected:       [3, 2, 1]
Got:            [3, 2, 1]
Head -> Tail:   3 ... 1
PASS ✅

[TEST 5]
Input:          [10, 20, 30, 40]
Expected:       [40, 30, 20, 10]
Got:            [40, 30, 20, 10]
Head -> Tail:   40 ... 10
PASS ✅


## 2. Rotate a Singly Linked List

Write a function that accepts a singly linked list and rotates it by n places to the right. If n is negative, rotate it abs(n) places to the left. Ensure the function is efficient and does not convert the linked list to a different data structure. 

Examples:

- 1->2->3->4->5 (rotate 2) result: 4->5->1->2->3
- 1->2->3->4->5 (rotate -2) result: 3->4->5->1->2

---
### Answers

The function below, `rotate_sll`, takes as input a singly-linked list (`SinglyLinkedList`), and an integer, `k`, which specifies the number of times to rotate the input list. Positive values of `k` rotate the list clockwise (right); negative anti-clockwise (left). This function operates in **$O(n)$ time** because, although the actual rotation involves updating a few references in $O(1)$, the entire list must be traversed in order to update the tail element's forward reference. This can be optimized by keeping track of the tail in the `SinglyLinkedList` class, but this is not necessarily a standard for this type of data structure. Regardless, traversal must be done to find the _rotation point_, which can be anywhere in the list up to $len(sll) - 1$, which is $O(n)$. The **space complexity is $O(1)$**, since all operations are done in-place (i.e., only "pointer" references are updated").

The function `rotate_sll_alt` is a re-write of the `rotate_sll` function, which performs all updates under a single loop. This is **not** intended to be used as the solution, but was included as an educational exercise.

In [4]:
def rotate_sll(sll: SinglyLinkedList[T], k: int):
    '''
    Rotates a singly-linked list in-place by k places.
    Negative values of k rotate left; Positive right.
    Time Complexity: O(n) since we must traverse the entire list once
    Space Complexity: O(1) since no extra space is needed
    '''

    # SinglyLinkedList stores the size metadata, so this is O(1)
    n = len(sll)
    
    # No rotation or empty list
    if k == 0 or n == 0:
        return

    # For simplicity, clamp k to the length of the list
    k = k % n
    
    # Again, no rotation
    if k == 0:
        return
        
    # Re: rotation algorithm is a[-k:] + a[:-k]
    r = n - k

    new_tail = sll.head
    for _ in range(r - 1):
        new_tail = new_tail.next

    # Set new head and ensure new tail does not have any forward references
    new_head = new_tail.next
    new_tail.next = None

    # Continue traversing until we find the current tail
    tail = new_head
    for _ in range(r, n - 1):
        tail = tail.next

    # Update forward references
    # Current tail -> current head
    # Current head -> new head
    tail.next = sll.head
    sll.head = new_head

def rotate_sll_alt(sll: SinglyLinkedList[T], k: int):
    '''
    An alternative re-write of `rotate_sll` which uses a single loop.
    Rotates a singly-linked list in-place by k places.
    Negative values of k rotate left; Positive right.
    Time Complexity: O(n) since we must traverse the list
    Space Complexity: O(1) since no extra space is needed
    '''

    # SinglyLinkedList stores the size metadata, so this is O(1)
    n = len(sll)
    
    # No rotation or empty list
    if k == 0 or n == 0:
        return

    # For simplicity, clamp k to the length of the list
    k = k % n
    
    # Again, no rotation
    if k == 0:
        return
        
    # Re: rotation algorithm is a[-k:] + a[:-k]
    r = n - k
    
    # Loop until the rotation point
    # and set new tail
    curr = sll.head
    _head = sll.head
    _tail = None
    for i in range(n - 1): # O(n)
        if i == r - 1:
            _tail = curr
            _head = curr.next
        curr = curr.next
    curr.next = sll.head # set current "tail's" next to the current head
    sll.head = _head # update the list's head to the rotated head
    _tail.next = None # ensure new "tail" has no next references 
            

In [5]:
def test_rotate_sll():
    test_cases = [
        # Format: (initial_list, k, expected_list)
        ([1, 2, 3, 4, 5], 2, [4, 5, 1, 2, 3]),   # Rotate right by 2
        ([1, 2, 3, 4, 5], -2, [3, 4, 5, 1, 2]),  # Rotate left by 2
        ([1, 2, 3], 0, [1, 2, 3]),               # No rotation
        ([], 3, []),                             # Empty list
        ([1], 10, [1]),                          # Single element, any k
        ([1, 2], 2, [1, 2]),                     # k == len(list)
        ([1, 2, 3, 4], 6, [3, 4, 1, 2]),         # k > len(list)
    ]

    for idx, (lst, k, expected) in enumerate(test_cases, 1):
        sll = SinglyLinkedList[int]()
        for val in lst:
            sll.append(val)
        
        rotate_sll(sll, k)
        result = sll.to_list()
        success = result == expected
        
        print(f"\nTest {idx}: k={k}, input={lst}")
        print(f"Expected: {expected}")
        print(f"Got     : {result}")
        print(f"PASS ✅" if success else "FAIL ❌")

# Run the test function
test_rotate_sll()



Test 1: k=2, input=[1, 2, 3, 4, 5]
Expected: [4, 5, 1, 2, 3]
Got     : [4, 5, 1, 2, 3]
PASS ✅

Test 2: k=-2, input=[1, 2, 3, 4, 5]
Expected: [3, 4, 5, 1, 2]
Got     : [3, 4, 5, 1, 2]
PASS ✅

Test 3: k=0, input=[1, 2, 3]
Expected: [1, 2, 3]
Got     : [1, 2, 3]
PASS ✅

Test 4: k=3, input=[]
Expected: []
Got     : []
PASS ✅

Test 5: k=10, input=[1]
Expected: [1]
Got     : [1]
PASS ✅

Test 6: k=2, input=[1, 2]
Expected: [1, 2]
Got     : [1, 2]
PASS ✅

Test 7: k=6, input=[1, 2, 3, 4]
Expected: [3, 4, 1, 2]
Got     : [3, 4, 1, 2]
PASS ✅


## 3. Linked List Union

Write an efficient function that takes two linked lists as input and returns a **new** linked list containing their union (all distinct elements from both lists).

- Ensure the implementation is as space and time efficient as possible
- The lists are not necessarily sorted
- The order of the output is not significant
- Do not use built-in union operators or functions

For example:

`[2, 10, 5, 3, 4]` and `[4, 7, 8, 3, 11]` has a union of `[2, 10, 3, 4, 5, 7, 8, 11]`

---
### Answers

The below `linked_list_union` function performs the linked list union operation in **$O(n)$ (avg) time and space complexities**. Since each function utilizes a hash map to check for the visit of a node with a particular value (deduplication), the average lookup time for a value is $O(1)$, with a worst case time of $O(n^2)$ in the unlikely scenario that there are hash collisions. The space complexity across all implementation is due to (minimally) the space required for the hash map to store unique values of traversed nodes, where in the worst case all values are unique, requiring $O(n)$ space.

The intended solution to this problem is the `linked_list_union` function, but for educational purposes different implementations have been made. The following are the nuances between each implementation:

- `linked_list_union` uses a purely iterative method to compute the result; there is no risk of recursion depth limitations, nor is there any extra space required for a call stack or new list. Forward reference mutations of node object are made in-place, enabling storage efficiency, and the resulting list simply points to the head node. This is the most efficient solution.
- `linked_list_union_recursive` uses a recursive implmentation to the prior function, and performs similarly in terms of time. However, it requires $O(n)$ extra space compared to the prior for the call stack. Furthermore, there is a limitation on the recursion depth, which therefore limits the length of the list that can be input to it. It uses tail recursion however, so programming languages optimized for this should not face this overhead limitation; Python does not do so.
- `linked_list_nondestructive` implements a more concise version of the `linked_list_union` function, using iteration to compute the hash map of seen values for each list and appending only unique results to an entirely new list. This implementation requires twice the space as the other implementations, but does not have the recurion depth limitation of `linked_list_union_recursive`. Furthermore, unlike the others this implementation runs in $O(n^2)$ time, since each iteration requires traversal of the new list to append unique nodes. It offers value if the input lists and their constituent nodes should not be mutated. Testing showed that this is the least efficient solution, almost twice as slow as the previous two. Optimization may be made by including a tail reference in the `SinglyLinkedList` implementation, but this is not a standard requirement.

In [6]:
from collections import defaultdict

def linked_list_union(l1: SinglyLinkedList[T], l2: SinglyLinkedList[T]) -> SinglyLinkedList[T]:
    '''
    Returns a new SinglyLinkedList containing all distinct elements from l1 and l2.
    This function uses iteration to traverse the lists and skip (drop) duplicates.
    Time Complexity: O(n), since all values of the lists must be traversed.
    Space Complexity: O(n), since extra space is needed for the hash map.
    '''
    seen = defaultdict(bool)
    result = SinglyLinkedList[T]()

    # Helper pointer to track the tail of the result
    tail: Optional[SLLNode[T]] = None

    # Iterate over both lists in order
    for lst in (l1, l2):
        curr = lst.head
        while curr:
            next_node = curr.next  # store next
            curr.next = None       # detach node
            if not seen[curr.value]:
                seen[curr.value] = True
                result._size += 1
                if tail:
                    tail.next = curr
                else:
                    result._head = curr
                tail = curr
            curr = next_node

    return result


def linked_list_union_recursive(l1: SinglyLinkedList[T], l2: SinglyLinkedList[T]) -> SinglyLinkedList[T]:
    '''
    Returns a new SinglyLinkedList containing all distinct elements from l1 and l2.
    Uses recursion to skip duplicate nodes.
    Time Complexity: O(n), since we must traverse all elements of the lists (worst case O(n^2) with hash collisions).
    Space Complexity: O(n), since extra space is needed for the hash map, as well as the call stack for recursion.
    '''
    seen = defaultdict(bool)
    result = SinglyLinkedList[T]()

    def process(node: Optional[SLLNode[T]], tail: Optional[SLLNode[T]]) -> Optional[SLLNode[T]]:
        """Recursively attach nodes whose values are not seen, updating tail and result._size."""
        if node is None:
            return tail

        next_node = node.next
        node.next = None  # detach safely

        if seen[node.value]:
            return process(next_node, tail)
        else:
            seen[node.value] = True
            result._size += 1
            if tail:
                tail.next = node
            else:
                result._head = node  # first unique node becomes head
            tail = node
            return process(next_node, tail)

    tail: Optional[SLLNode[T]] = None
    tail = process(l1.head, tail)
    tail = process(l2.head, tail)

    return result

def linked_list_union_nondestructive(l1: SinglyLinkedList[T], l2: SinglyLinkedList[T]) -> SinglyLinkedList[T]:
    """
    Returns a new SinglyLinkedList containing all distinct elements from l1 and l2. This does not mutate the elements
    of the input lists, and as a result is non-destructive (with the tradeoff of greater memory consumption).
    Time Complexity: O(n^2), since all elements of the lists must be traversed, and appending requires O(n) time as well.
    Space Complexity: O(n), since extra space is needed for the hash map and new list
    """
    seen = defaultdict(bool)
    union_list = SinglyLinkedList[T]()

    # Helper to append unseen values to the union list
    def append_unique(node):
        while node: # O(n)
            if not seen[node.value]:
                union_list.append(node.value) # O(n), since traversal to tail
                seen[node.value] = True
            node = node.next

    append_unique(l1.head)
    append_unique(l2.head)

    return union_list

In [7]:
def test_linked_list_unions():
    test_cases = [
        # (l1 values, l2 values, expected union)
        ([], [], []),
        ([1, 2, 3], [], [1, 2, 3]),
        ([], [1, 2, 3], [1, 2, 3]),
        ([2, 10, 5, 3, 4], [4, 7, 8, 3, 11], [2, 10, 5, 3, 4, 7, 8, 11]),
        ([1, 1, 2], [2, 3, 3], [1, 2, 3]),
        ([5, 6, 7], [5, 6, 7], [5, 6, 7]),
    ]

    union_functions = [
        ("Iterative", linked_list_union),
        ("Recursive", linked_list_union_recursive),
        ("Non-destructive", linked_list_union_nondestructive)
    ]

    for name, func in union_functions:
        print(f"Testing {name} version...")
        for i, (l1_vals, l2_vals, expected) in enumerate(test_cases, start=1):
            # Build fresh linked lists for each test
            l1 = SinglyLinkedList[T]()
            l2 = SinglyLinkedList[T]()
            for v in l1_vals:
                l1.append(v)
            for v in l2_vals:
                l2.append(v)

            result = func(l1, l2)

            result_list = result.to_list()
            result_set = set(result_list)
            expected_set = set(expected)

            # Order not important, check content only
            pass_fail = "PASS ✅" if result_set == expected_set else "FAIL ❌"

            print(
                f"Test {i}: l1={l1_vals}, l2={l2_vals} -> "
                f"union={result_list}, expected (set)={expected_set} "
                f"{pass_fail}"
            )
        print("\n")

test_linked_list_unions()

Testing Iterative version...
Test 1: l1=[], l2=[] -> union=[], expected (set)=set() PASS ✅
Test 2: l1=[1, 2, 3], l2=[] -> union=[1, 2, 3], expected (set)={1, 2, 3} PASS ✅
Test 3: l1=[], l2=[1, 2, 3] -> union=[1, 2, 3], expected (set)={1, 2, 3} PASS ✅
Test 4: l1=[2, 10, 5, 3, 4], l2=[4, 7, 8, 3, 11] -> union=[2, 10, 5, 3, 4, 7, 8, 11], expected (set)={2, 3, 4, 5, 7, 8, 10, 11} PASS ✅
Test 5: l1=[1, 1, 2], l2=[2, 3, 3] -> union=[1, 2, 3], expected (set)={1, 2, 3} PASS ✅
Test 6: l1=[5, 6, 7], l2=[5, 6, 7] -> union=[5, 6, 7], expected (set)={5, 6, 7} PASS ✅


Testing Recursive version...
Test 1: l1=[], l2=[] -> union=[], expected (set)=set() PASS ✅
Test 2: l1=[1, 2, 3], l2=[] -> union=[1, 2, 3], expected (set)={1, 2, 3} PASS ✅
Test 3: l1=[], l2=[1, 2, 3] -> union=[1, 2, 3], expected (set)={1, 2, 3} PASS ✅
Test 4: l1=[2, 10, 5, 3, 4], l2=[4, 7, 8, 3, 11] -> union=[2, 10, 5, 3, 4, 7, 8, 11], expected (set)={2, 3, 4, 5, 7, 8, 10, 11} PASS ✅
Test 5: l1=[1, 1, 2], l2=[2, 3, 3] -> union=[1, 2, 3

In [8]:
import timeit
from typing import Callable, List, Tuple

def benchmark_linked_list_unions(factories: List[Callable[[], Tuple[SinglyLinkedList[int], SinglyLinkedList[int]]]],
                                 iterations: int = 100):
    """
    Benchmark multiple union functions using pre-defined factories.

    Args:
        factories: List of callables, each returning a tuple of (l1, l2)
        iterations: Number of repetitions per factory
    """
    functions = [
        ("iterative", linked_list_union),
        ("recursive", linked_list_union_recursive),
        ("nondestructive", linked_list_union_nondestructive),
    ]

    results = []

    for name, func in functions:
        total_time = 0.0
        for factory in factories:
            def wrapper():
                l1, l2 = factory()
                func(l1, l2)

            total_time += timeit.timeit(wrapper, number=iterations)
        avg_time = total_time / (iterations * len(factories))
        results.append((name, avg_time))

    # Sort by fastest
    results.sort(key=lambda x: x[1])

    print(f"Number of factories: {len(factories)}")
    print(f"Average execution times over {iterations} iterations per factory:")
    for name, t in results:
        print(f"{name:15s}: {t*1000:.3f} ms")


# -----------------------------
# Example factory with seeded RNG
# -----------------------------
import random

def seeded_random_factory(seed: int = 42, size: int = 1000, overlap_frac: float = 0.3):
    """
    Returns a factory function that generates two linked lists with controlled overlap and seeded RNG.
    """
    def factory() -> Tuple[SinglyLinkedList[int], SinglyLinkedList[int]]:
        random.seed(seed)  # deterministic
        n_overlap = int(size * overlap_frac)
        n_unique = size - n_overlap

        shared_vals = random.sample(range(size * 3), n_overlap)
        l1_unique = random.sample(range(size * 3), n_unique)
        l2_unique = random.sample(range(size * 3), n_unique)

        l1_vals = l1_unique + shared_vals
        l2_vals = l2_unique + shared_vals
        random.shuffle(l1_vals)
        random.shuffle(l2_vals)

        l1 = SinglyLinkedList[int]()
        l2 = SinglyLinkedList[int]()
        for v in l1_vals:
            l1.append(v)
        for v in l2_vals:
            l2.append(v)
        return l1, l2

    return factory


# -----------------------------
# Example usage
# -----------------------------
factories = [seeded_random_factory(seed=i) for i in range(3)]  # 3 different seeded factories
benchmark_linked_list_unions(factories)

Number of factories: 3
Average execution times over 100 iterations per factory:
recursive      : 15.098 ms
iterative      : 15.360 ms
nondestructive : 28.699 ms


## 4. Flatten a Binary Tree

Write two functions to convert a valid binary tree, flatten it into a linked list and return it.

- The first function should flatten it using preorder traversal
- The second function should flatten it using postorder traversal

Write it as time and space efficient as possible.

For example, given the following tree

![Q4 Binary Tree](./images/dsa_assignment_04_binary_tree.png)

- Preorder linked list: 1->2->4->5->3->6
- Postorder linked list: 4->5->2->6->3->1

---
### Answers

The below functions flatten an input binary tree, `BinaryTree`, into a `SinglyLinkedList` linked-list. The intended solutions for this question are the `flatten_preorder_time_optimized` and `flatten_postorder_time_optimized` functions. Below is an overview of each function:

- `flatten_preorder` is a naiive implementation of the preorder traversal of the Binary Tree object. It utilizes the built-in `preorder` method of the `BinaryTree` class, which simply traverses the tree recursively, visiting `node` -> `node.left` -> `node.right`. It takes as an argument a callable to "process" each visited node's value; in this case each value is appended to a `SinglyLinkedList`. This append operation turns out to be the major bottleneck in the resulting time complexity, since it is $O(n)$ in and of itself.
    - Time Complexity: $O(n^2)$ since traversal takes $O(n)$ and for each traversal, appends to the singly-linked list is $O(n)$
    - Space Complexity: $O(n)$ since $n$ extra space is required for the new singly-linked list
- `flatten_postorder` is a naiive implementation of the postorder traversal of the Binary Tree object. Similarly to the above, it uses the build-in `postorder` method of the BinaryTree object to visit each node in postorder `node.left` -> `node.right` -> `node`. At each visit, a function is called to append the node's value to a `SinglyLinkedList`.
    - Time Complexity: $O(n^2)$ since traversal takes $O(n)$ and for each traversal, appends to the singly-linked list is $O(n)$
    - Space Complexity: $O(n)$ since $n$ extra space is required for the new singly-linked list

Now, it is noted that we must traverse all nodes of the tree in all cases, which is $O(n)$; thus the only way to improve the overall time compexity is by optimizing the singly-linked list append operation. The functions `flatten_preorder_time_optimized` and `flatten_postorder_time_optimized` keep track of the tail node in the singly-linked list, making append operations $O(1)$.

- `flatten_preorder_time_optimized`:
    - Time Complexity: $O(n)$ to traverse the binary tree
    - Space Complexity: $O(n)$ since $n$ extra space is required for the new singly-linked list
- `flatten_postorder_time_optimized`:
    - Time Complexity: $O(n)$ to traverse the binary tree
    - Space Complexity: $O(n)$ since $n$ extra space is required for the new singly-linked list


Additionally, there is a further optimization (albeit a destructive one), which also enables $O(1)$ space complexity for **preorder** traversal. This is a modification of the _Morris trick_, which modifies the Binary Tree in-place to produce a preorder "linked-list" by removing all left node references. This is not a true linked list, and is why this solution has been omitted.

- `flatten_postorder_space_optimized`:
    - Time Complexity: $O(n)$ to traverse the binary tree
    - Space Complexity: $O(1)$ since no extra space is required (in-place operation)



In [9]:
def flatten_preorder(bt: BinaryTree[C]) -> SinglyLinkedList[C]:
    '''
    Takes as input a binary tree and flattens it into a singly-linked list,
    using preorder traversal.
    Time Complexity: O(n^2) since traversal takes O(n) and for each traversal, O(n) traversals
    of the linked list are required for each append operation
    Space Complexity: O(n) since n extra space is required for the new singly-linked list
    '''
    sll = SinglyLinkedList()
    bt.preorder(lambda x: sll.append(x)) # O(n) * O(n)
    return sll
    
def flatten_postorder(bt: BinaryTree[C]) -> SinglyLinkedList[C]:
    '''
    Takes as input a binary tree and flattens it into a singly-linked list,
    using postorder traversal.
    Time Complexity: O(n^2) since traversal takes O(n) and for each traversal, O(n) traversals
    of the linked list are required for each append operation
    Space Complexity: O(n) since n extra space is required for the new singly-linked list
    '''
    sll = SinglyLinkedList()
    bt.postorder(lambda x: sll.append(x)) # O(n) * O(n)
    return sll

 
def flatten_preorder_time_optimized(bt: BinaryTree[C]) -> SinglyLinkedList[C]:
    '''
    Takes as input a binary tree and flattens it into a singly-linked list,
    using preorder traversal. This version optimizes for time by keeping track
    of the tail node, so appends only take O(1) instead of O(n) with traversals.
    Time Complexity: O(n) since traversal takes O(n) and appends only take O(1)
    of the linked list are required for each append operation
    Space Complexity: O(n) since n extra space is required for the new singly-linked list
    '''
    sll = SinglyLinkedList()
    tail = None
    
    def appendNode(value: C):
        nonlocal tail
        new_node = SLLNode(value)
        if tail is None:
            tail = new_node
            sll.head = tail
            return
        tail.next = new_node
        tail = tail.next # advance tail
        
    bt.preorder(appendNode) # O(n) * O(1)
    return sll
    
def flatten_postorder_time_optimized(bt: BinaryTree[C]) -> SinglyLinkedList[C]:
    '''
    Takes as input a binary tree and flattens it into a singly-linked list,
    using postorder traversal. This version optimizes for time by keeping track
    of the tail node, so appends only take O(1) instead of O(n) with traversals.
    Time Complexity: O(n) since traversal takes O(n) and appends only take O(1)
    of the linked list are required for each append operation
    Space Complexity: O(n) since n extra space is required for the new singly-linked list
    '''
    sll = SinglyLinkedList()
    tail = None
    
    def appendNode(value: C):
        nonlocal tail
        new_node = SLLNode(value)
        if tail is None:
            tail = new_node
            sll.head = tail
            return
        tail.next = new_node
        tail = tail.next # advance tail
        
    bt.postorder(appendNode) # O(n) * O(1)
    return sll
    
def flatten_preorder_space_optimized(bt: BinaryTree[C]) -> BinaryTree[C]:
    '''
    Flatten the binary tree into a "right-only" linked list in-place using preorder traversal.
    This version optimizes for space by modifying the binary tree into a "false" linked-list, 
    by keeping only "right" references. This is a modified version of the "Morris trick".
    Time Complexity: O(n) amortized for tree traversal
    Space Complexity: O(1) since no extra space is needed (in-place)
    '''
    node = bt.root
    while node:
        if node.left:
            # Find the rightmost node of the left subtree
            rightmost = node.left
            while rightmost.right:
                rightmost = rightmost.right
            # Rewire connections
            rightmost.right = node.right
            node.right = node.left
            node.left = None
        node = node.right

    return bt


In [10]:
import copy

def test_flatten_binary_tree():
    # Each test case: (input tuple for tree, expected preorder list, expected postorder list)
    test_cases = [
        ((), [], []),  # empty tree
        ((1,), [1], [1]),  # single node
        ((4, 2, 6, 1, 3, 5, 7), [4, 2, 1, 3, 6, 5, 7], [1, 3, 2, 5, 7, 6, 4]),  # balanced
        ((5, 4, 3, 2, 1), [5, 4, 3, 2, 1], [1, 2, 3, 4, 5]),  # left-heavy
        ((1, 2, 3, 4, 5), [1, 2, 3, 4, 5], [5, 4, 3, 2, 1]),  # right-heavy
        ((5, 3, 7, 3, 5, 8), [5, 3, 3, 7, 5, 8], [3, 3, 5, 8, 7, 5]),  # duplicates
    ]
    for idx, (input_vals, expected_pre, expected_post) in enumerate(test_cases, start=1):
        print(f"\nTest Case {idx}: input={input_vals}")
        bt = BinaryTree[int]()
        for v in input_vals:
            bt.insert(v)
        print("Binary Tree structure:")
        print(bt)
        print(bt.pretty_print())

        # Flatten using preorder
        sll_pre = flatten_preorder(bt)
        pre_list = sll_pre.to_list()
        print(f"Preorder flattened: {pre_list}, expected: {expected_pre}")
        assert pre_list == expected_pre, f"Test {idx} Preorder Failed"
        
        # Flatten using preorder (time optimized)
        sll_pre = flatten_preorder_time_optimized(bt)
        pre_list = sll_pre.to_list()
        print(f"Preorder flattened (Time Optimized): {pre_list}, expected: {expected_pre}")
        assert pre_list == expected_pre, f"Test {idx} Preorder (Time Optimized) Failed"
        
        # Flatten using preorder (space optimized)
        _bt = copy.deepcopy(bt)
        flatten_preorder_space_optimized(_bt)
        pre_list = _bt.to_list()
        print(f"Preorder flattened (Space Optimized): {pre_list}, expected: {expected_pre}")
        assert pre_list == expected_pre, f"Test {idx} Preorder (Time Optimized) Failed"

        # Flatten using postorder
        sll_post = flatten_postorder(bt)
        post_list = sll_post.to_list()
        print(f"Postorder flattened: {post_list}, expected: {expected_post}")
        assert post_list == expected_post, f"Test {idx} Postorder Failed"
        
        # Flatten using postorder (time optimized)
        sll_post = flatten_postorder_time_optimized(bt)
        post_list = sll_post.to_list()
        print(f"Postorder flattened (Time Optimized): {post_list}, expected: {expected_post}")
        assert post_list == expected_post, f"Test {idx} Postorder (Time Optimized) Failed"
        
        print(f"Test Case {idx} passed ✅")

    print("\nAll flatten tests passed successfully!")

# Run the tests
test_flatten_binary_tree()


Test Case 1: input=()
Binary Tree structure:
BinaryTree([])
<empty>
Preorder flattened: [], expected: []
Preorder flattened (Time Optimized): [], expected: []
Preorder flattened (Space Optimized): [], expected: []
Postorder flattened: [], expected: []
Postorder flattened (Time Optimized): [], expected: []
Test Case 1 passed ✅

Test Case 2: input=(1,)
Binary Tree structure:
BinaryTree([1])
┌1┐
Preorder flattened: [1], expected: [1]
Preorder flattened (Time Optimized): [1], expected: [1]
Preorder flattened (Space Optimized): [1], expected: [1]
Postorder flattened: [1], expected: [1]
Postorder flattened (Time Optimized): [1], expected: [1]
Test Case 2 passed ✅

Test Case 3: input=(4, 2, 6, 1, 3, 5, 7)
Binary Tree structure:
BinaryTree([1, 2, 3, 4, 5, 6, 7])
         ┌4┐         
   ┌2┐         ┌6┐   
┌1┐   ┌3┐   ┌5┐   ┌7┐
Preorder flattened: [4, 2, 1, 3, 6, 5, 7], expected: [4, 2, 1, 3, 6, 5, 7]
Preorder flattened (Time Optimized): [4, 2, 1, 3, 6, 5, 7], expected: [4, 2, 1, 3, 6, 5, 7]
P